In [18]:
import os
import pandas as pd
from pprint import pprint
from dataclasses import dataclass

In [21]:
@dataclass
class PipelineConfig:
    prompt_dir: str = "data/prompt_output"
    llm_output_dir: str = "data/llm_output"
    result_dir: str = "data/result"
    target_tasks: list = ("NLI", "SA")
    provider: str = "gemini"
    models: str = ("gemini-1.5-flash", "gemini-1.5-pro")
    example_num: int = 100


pipeline_config = PipelineConfig()

pprint(pipeline_config, indent=4)

PipelineConfig(prompt_dir='data/prompt_output',
               llm_output_dir='data/llm_output',
               result_dir='data/result',
               target_tasks=('NLI', 'SA'),
               provider='gemini',
               models=('gemini-1.5-flash', 'gemini-1.5-pro'),
               example_num=100)


### Step 1
`src.prompt_creator.create_prompt` モジュールを使用して、実験に使用するプロンプトを生成する。

In [2]:
from src.prompt_creator.create_prompt import PromptCreator

/Users/satoujouichirou/Repositories/pj-b-research-prompt-injection-attack/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
promptcreator = PromptCreator("NLI", "NLI", 100)

for method in ["combined-attack", "context-ignoring", "escape-characters", "fake-completion", "naive-attack"]:
    os.makedirs(os.path.join(pipeline_config.prompt_dir, method), exist_ok=True)
    prompt= promptcreator.create_prompt(method)
    prompt_df = pd.DataFrame(prompt)
    prompt_df.to_csv(os.path.join(pipeline_config.prompt_dir, method, f"NLI_NLI_100.csv"), index=False)

target task: NLI, injected task: NLI, example_num: 100
target task: NLI, injected task: NLI, example_num: 100
target task: NLI, injected task: NLI, example_num: 100
target task: NLI, injected task: NLI, example_num: 100
target task: NLI, injected task: NLI, example_num: 100


### Step 2
`src.llm_processor.llm_processor` モジュールを使用して、LLMに回答を出力させる

In [7]:
from src.llm_processor.llm_processor import LLM

In [8]:
llm_input_dir = "data/prompt_output"
llm_output_dir = "data/llm_output"

In [13]:
for model in ["gemini-1.5-flash", "gemini-1.5-pro"]:
    llm = LLM(provider="gemini", model=model)
    for method in ["combined-attack", "context-ignoring", "escape-characters", "fake-completion", "naive-attack"]:
        print(f"Processing {model} {method}")
        os.makedirs(os.path.join(llm_output_dir, model, method), exist_ok=True)
        prompt = pd.read_csv(os.path.join(llm_input_dir, method, "NLI_NLI_100.csv"))
        llm_output = llm.create_llm_outputs(prompt)
        llm_output.to_csv(os.path.join(llm_output_dir, model, method, f"NLI_NLI_100.csv"), index=False)

Processing gemini-1.5-flash combined-attack
Processing gemini-1.5-flash context-ignoring
Processing gemini-1.5-flash escape-characters
Processing gemini-1.5-flash fake-completion
Processing gemini-1.5-flash naive-attack
Processing gemini-1.5-pro combined-attack
Processing gemini-1.5-pro context-ignoring
Processing gemini-1.5-pro escape-characters
Processing gemini-1.5-pro fake-completion
Processing gemini-1.5-pro naive-attack


### Step 3
`src.evaluator.attack_benchmark` を用いて攻撃の評価をする

In [22]:
from src.evaluator.attack_benchmark import (
    asv,
    mr,
    pna
)

In [16]:
llm_output_dir = "data/llm_output"
results_dir = "output"

In [17]:
for model in ["gemini-1.5-flash", "gemini-1.5-pro"]:
    for method in ["combined-attack", "context-ignoring", "escape-characters", "fake-completion", "naive-attack"]:
        os.makedirs(os.path.join(results_dir, model, method), exist_ok=True)
        print(f"Evaluating {model} {method}")
        
        llm_output_df = pd.read_csv(os.path.join(llm_output_dir, model, method, "NLI_NLI_100.csv"))
        output_df = pd.DataFrame()
        output_df["ASV"] = asv.asv(llm_output_df)["ASV"]
        output_df["MR"] = mr.mr(llm_output_df)["MR"]
        pna_df = pna.pna(llm_output_df)
        output_df["PNA-T"] = pna_df["PNA-T"]
        output_df["PNA-I"] = pna_df["PNA-I"]
        
        output_df.to_csv(os.path.join(results_dir, model, method, "NLI_NLI_100.csv"), index=False)

Evaluating gemini-1.5-flash combined-attack
Evaluating gemini-1.5-flash context-ignoring
Evaluating gemini-1.5-flash escape-characters
Evaluating gemini-1.5-flash fake-completion
Evaluating gemini-1.5-flash naive-attack
Evaluating gemini-1.5-pro combined-attack
Evaluating gemini-1.5-pro context-ignoring
Evaluating gemini-1.5-pro escape-characters
Evaluating gemini-1.5-pro fake-completion
Evaluating gemini-1.5-pro naive-attack
